**Sun 2017-09-10 8pm:** Focusing on ProAnti, using multiple rule_and_delay_period values, as well as multiple post_target_period_values.  Seems to be working fine. Should to set up a parameter space search.


<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [33]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('https://sites.google.com/site/brodylabhome/files/make_table_of_contents.js')
"""

In [10]:

using PyCall
using PyPlot
using ForwardDiff
using DiffBase
using MAT

pygui(true)

import Base.convert
convert(::Type{Float64}, x::ForwardDiff.Dual) = Float64(x.value)
function convert(::Array{Float64}, x::Array{ForwardDiff.Dual}) 
    y = zeros(size(x)); 
    for i in 1:prod(size(x)) 
        y[i] = convert(Float64, x[i]) 
    end
    return y
end

include("general_utils.jl")
include("hessian_utils.jl")

"""
We define functions to convert Duals, the variable types used by ForwardDiff, 
to Floats. This is useful if we want to print out the value of a variable 
(since print doesn't know how to Duals). Note that after being converted to a Float, no
differentiation by ForwardDiff can happen!  e.g. after
    x = convert(Float64, y)
ForwardDiff can still differentiate y, but it can't differentiate x
"""



"We define functions to convert Duals, the variable types used by ForwardDiff, \nto Floats. This is useful if we want to print out the value of a variable \n(since print doesn't know how to Duals). Note that after being converted to a Float, no\ndifferentiation by ForwardDiff can happen!  e.g. after\n    x = convert(Float64, y)\nForwardDiff can still differentiate y, but it can't differentiate x\n"

# Setup -- definitions of forwardModel() and backwardsModel()

These are functions that run arbitrary 0.5 * (1+tanh(x)) - style rate networks, either forwards in time, or backwards in time.

In [11]:
"""
o = g(z)    squashing tanh function, running from 0 to 1, is equal to 0.5 when input is 0.
"""
function g(z)
    return 0.5*tanh.(z)+0.5
end
    
"""
z = g^-1(o)    inverse of squashing tanh function, input must be in (0, 1), output is zero when passed 0.5.
"""
function ginverse(z)
    return 0.5*log.(z./(1-z))
end


"""
forwardModel(startU; dt=0.01, tau=0.1, nsteps=100, input=[0.1, 0], noise=[], W=[0 -5;-5 0], 
init_add=0, start_add=0, const_add=0, sigma=0, gleak=1, U_rest=0, 
    do_plot=false, nderivs=0, difforder=0, clearfig=true, fignum=1, dUdt_mag_only=false,
    warn_if_unused_params=false)

Runs a tanh() style-network forwards in time, given its starting point, using simple Euler integration
    tau dU/dt = -U + W*V + I
    V = 0.5*tanh(U)+ 0.5

**PARAMETERS:**

startU     A column vector, nunits-by-1, indicating the values of U at time zero


**OPTIONAL PARAMETERS**

dt      Scalar, timestep size

tau     Scalar, in seconds

gleak   
        dUdt will have a term equal to gleak*(U_rest - U)
U_rest

nsteps  Number of timesteps to run, including time=0.

input   Either an nunits-by-1 vector, in which case inputs to each unit are constant
        across time, or a matrix, nunits-by-nsteps, indicating input for each unit at each timepoint.

W       Weight matrix, nunits-by-nunits

init_add    DEPRECATED: Vector or scalar that gets added to the input current at very first timestep.
            Deprecated because this made it dt-dependent. Replaced by start_add.

start_add   Vector or scalar that gets added, once, to the initial U[:,1], before the integration process begins.

const_add   Scalar that gets added to U after every timestep

sigma       After each timestep, add sigma*sqrt(dt)*randn() to each element of U

do_plot   Default false, if true, plots V of up to the first two dimensions

fignum     Figure number on which to plot

clrearfig  If true, the figure is first cleared, otherwise any plot ois overlaid

nderivs, difforder     Required for making sure function can create its own arrays and 
                       still be differentiated

dUdt_mag_only  If true, returns |dUdt|^2 from the first timestep only, then stops.

warn_if_unused_params     If true, pronts out a warning of some of the passed parameters are not used.

** RETURNS:**

Uend Vend       nunits-by-1 vectors representing the final values of U and V that were found.
U, V            nunits-by-nsteps matrices containing the full trajectories

"""
function forwardModel(startU; dt=0.01, tau=0.1, nsteps=100, input=[], noise=[], W=[0 -5;-5 0], 
    init_add=0, start_add=0, const_add=0, do_plot=false, nderivs=0, difforder=0, clearfig=true, fignum=1,
    dUdt_mag_only=false, sigma=0, g_leak=1, U_rest=0, theta=0, beta=1, 
    warn_if_unused_params=false, other_unused_params...)

    if warn_if_unused_params && length(other_unused_params)>0
        @printf("\n\n=== forwardModel warning, had unused params ")
        for k in keys(Dict(other_unused_params))
            @printf("%s, ", k)
        end
    end
    
    my_input = ForwardDiffZeros(size(input,1), size(input,2), nderivs=nderivs, difforder=difforder)
    for i=1:prod(size(input)); my_input[i] = input[i]; end
    input = my_input;
    
    nunits = length(startU)
    if size(startU,2) > size(startU,1)
        error("startU must be a column vector")
    end
    
    # --- formatting input ---
    if ~(typeof(input)<:Array) || prod(size(input))==1  # was a scalar
        input = input[1]*(1+ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder))
    elseif length(input)==0 # was the empty matrix
        input = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    elseif size(input,2)==1     # was a column vector
        input = input*(1+ForwardDiffZeros(1, nsteps, nderivs=nderivs, difforder=difforder))
    end    
    # --- formatting noise ---
    if ~(typeof(noise)<:Array) || prod(size(noise))==1  # was a scalar
        noise = noise*(1+ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder))
    elseif length(noise)==0 # was the empty matrix
        noise = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    elseif size(noise,2)==1     # was a column vector
        noise = noise*(1+ForwardDiffZeros(1, nsteps, nderivs=nderivs, difforder=difforder))
    end    
    
    U = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    V = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    
    if ~(typeof(W)<:Array); W = [W]; end

    W     = reshape(W, nunits, nunits)
    U     = reshape(U, nunits, nsteps)
    V     = reshape(V, nunits, nsteps)
    input = reshape(input, nunits, nsteps)
    noise = reshape(noise, nunits, nsteps)

    input[:,1] += init_add
    input      += const_add

    #@printf("size(U) is (%d,%d), and size(startU) is (%d,%d) and size(noise) is (%d,%d)", 
    #    size(U,1), size(U,2), size(startU,1), size(startU,2), size(noise,1), size(noise,2))
    # @printf("U[1]=%g, noise[1]=%g\n", startU, noise[1])
    U[:,1] = startU + noise[:,1] + start_add; # @printf("Resulting U=%g\n", U[1])
    V[:,1] = g((U[:,1]-theta)/beta); # @printf("Resulting V=%g\n", V[1])
    
    for i=2:nsteps
        dUdt = g_leak*(U_rest -U[:,i-1]) + W*V[:,i-1] + input[:,i-1]
        if dUdt_mag_only; return sum(dUdt.*dUdt); end;
        # @printf("dUdt=%g\n", dUdt[1])
        # @printf("i=%g\n", i)
        # @printf("noise[2]=%g\n", noise[2])
        U[:,i] = U[:,i-1] + (dt/tau)*dUdt + noise[:,i] + sigma*sqrt(dt)*randn(size(U,1),1)
        # @printf("Resulting U[2]=%g\n", U[2])
        V[:,i] = g((U[:,i]-theta)/beta)
        # @printf("Resulting V[2]=%g\n", V[2])
    end

    if do_plot
        figure(fignum)
        if length(startU)==1
            if clearfig; clf(); end;
            t = (0:nsteps-1)*dt
            plot(t, V[1,:], "b-")
            plot(t[1], V[1,1], "g.")
            plot(t[end], V[1,end], "r.")
            xlabel("t"); ylabel("V1"); ylim([-0.01, 1.01])
        elseif length(startU)>=2
            if clearfig; clf(); end;
            plot(V[1,:], V[2,:], "b-")
            plot(V[1,1], V[2,1], "g.")
            plot(V[1,end], V[2,end], "r.")
            xlabel("V1"); ylabel("V2"); 
            xlim([-0.01, 1.01]); ylim([-0.01, 1.01])
        end
    end

    return U[:,end], V[:,end], U, V
end


"""
backwardsModel(endU; dt=0.01, tau=0.1, nsteps=100, input=[0],noise=[],  W=[0 -5;-5 0], 
    do_plot=false, nderivs=0, difforder=0, clearfig=true, fignum=1, tol=1e-15, start_eta=10)

Runs a tanh() style-network BACKWARDS in time, given its ending point, by making a backwards
guess at each timepoint and then using Hessian minimization to find the backwards vector that correctly
leads to the current timestep value.  Uses forwardModel() . The forwards equations are:

    tau dU/dt = -U + W*V + I
    V = 0.5*tanh(U)+ 0.5

**PARAMETERS:**

endU     A column vector, nunits-by-1, indicating the values of U at time=end


**OPTIONAL PARAMETERS:**

dt      Scalar, timestep size

tau     Scalar, in seconds

nsteps  Number of timesteps to run, including time=0.

input   Either an nunits-by-1 vector, in which case inputs to each unit are constant
        across time, or a matrix, nunits-by-nsteps, indicating input for each unit at each timepoint.

W       Weight matrix, nunits-by-nunits

do_plot   Default false, if true, plots V of up to the first two dimensions

tol       Tolerance in the minimization procedure for finding each backwards timestep. Passed on
          to trust_region_Hessian_minimization()

start_eta   Passed on to trust_region_Hessian_minimization()

fignum     Figure number on which to plot

clrearfig  If true, the figure is first cleared, otherwise any plot ois overlaid

nderivs, difforder     Required for making sure function can create its own arrays and 
                       still be differentiated



** RETURNS:**

Ustart Vstart   nunits-by-1 vectors representing the starting values of U and V that were found.
U, V            nunits-by-nsteps matrices containing the full trajectories
costs           1-by-nsteps vector with the final cost from the minimization procedure for each
                timestep. This is the squared difference between the U[t+1] produced by the U[t] 
                guess and the actual U[t+1]

"""
function backwardsModel(endU; nsteps=100, start_eta=10, tol=1e-15, maxiter=400, 
    do_plot=false, init_add=0, start_add=0, dt=0.01, 
    input=[], noise=[], nderivs=0, difforder=0, clearfig=false, fignum=1, params...)    

    nunits = length(endU)

    # --- formatting input ---
    if ~(typeof(input)<:Array) || prod(size(input))==1  # was a scalar
        input = input[1]*(1+ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder))
    elseif length(input)==0 # was the empty matrix
        input = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    elseif size(input,2)==1     # was a column vector
        input = input*(1+ForwardDiffZeros(1, nsteps, nderivs=nderivs, difforder=difforder))
    end    
    # --- formatting noise ---
    if ~(typeof(noise)<:Array)  # was a scalar
        noise = noise*(1+ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder))
    elseif length(noise)==0 # was the empty matrix
        noise = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    elseif size(noise,2)==1     # was a column vector
        noise = noise*(1+ForwardDiffZeros(1, nsteps, nderivs=nderivs, difforder=difforder))
    end    
    
    function J(U1, U2; nderivs=0, difforder=0, noise=[], inputs=[], pars...)
        U2hat = forwardModel(U1; nsteps=2, noise=noise, input=input, nderivs=nderivs, difforder=difforder, pars...)[1]
        U2hat = U2hat
        DU = U2hat - U2
    
        return sum(DU.*DU)
    end
    
    if length(noise)==0
        noise = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    end

    U = ForwardDiffZeros(nunits, nsteps, nderivs=nderivs, difforder=difforder)
    U = reshape(U, nunits, nsteps)
    costs = ForwardDiffZeros(nsteps, 1, nderivs=nderivs, difforder=difforder)    
    
    U[:,end] = endU
    for i=(nsteps-1):-1:1
        if i==1
            my_init_add = init_add
            my_start_add = start_add
        else
            my_init_add = 0
            my_start_add = 0
        end
                
        U[:,i], costs[i] = trust_region_Hessian_minimization(U[:,i+1], 
            (x) -> J(x, U[:,i+1]; nderivs=length(endU), difforder=2, 
            input=input[:,i:i+1], noise = noise[:,i:i+1], 
            init_add=my_init_add, start_add=my_start_add, params...); 
            verbose=false, start_eta=start_eta, tol=tol, maxiter=maxiter)
        if i>1; U[:,i] += noise[:,i]; end
    end
    
    
    V = g(U)
    
    if do_plot
        figure(fignum)   
        if typeof(params)<:Array; params = Dict(params); end;
        if haskey(params, :dt);     dt     = params[:dt];     end
        if haskey(params, :nsteps); nsteps = params[:nsteps]; end
        if length(endU)==1
            if clearfig; clf(); end;
            t = (0:nsteps-1)*dt
            plot(t, V[1,:], "m-")
            plot(t[1], V[1,1], "go")
            plot(t[end], V[1,end], "ro")            
            ylim([-0.01, 1.01])
        elseif length(endU)>=2
            if clearfig; clf(); end;            
            plot(V[1,:], V[2,:], "m-")
            plot(V[1,1], V[2,1], "go")
            plot(V[1,end], V[2,end], "ro")
            xlim([-0.01, 1.01]); ylim([-0.01, 1.01])
        end
    end
    
    return U[:,1], V[:,1], U, V, costs
end

backwardsModel

### Testing forward and backwards models with only 1 dimension

In [12]:
figure(1); clf();
params = Dict(:noise => [0.1], :W => [-2], :nsteps=>10, :start_add=>-1.9)
Uend = forwardModel([1.1]; do_plot=true, params...)[1]
Ustart = backwardsModel(Uend; do_plot=true, tol=1e-30, params...)[1]
@printf("Ustart came back as %g\n", Ustart[1])

Ustart came back as 1.1


### Testing forward and backwards models now with 2 dimensions

In [13]:
nsteps=50
params = Dict(:noise =>0.03*randn(2,nsteps) + [0.1,0]*ones(1,nsteps), :W => [0 -5; -5 0], :nsteps=>nsteps)

Uend, Vend, U, V              = forwardModel([0.1,0.1]; do_plot=true, params...);
Ustart, Vstart, bU, bV, costs = backwardsModel(Uend; do_plot=true, tol=1e-30, params...)

@printf("Ustart came back as : "); print_vector_g(Ustart); print("\n")

Ustart came back as : [0.1, 0.1]


# Exploring dt-dependence of gradients and hessian

In [14]:
# srand(111)
startU=randn(100,2)-3
startU=randn(100,2)-3
sigma = 0

# startU=0.1*randn(100,2)-3
# startU=zeros(100,2)-3


dt = 0.02
t = 0:dt:1
tau = 0.1
nsteps = length(t)
t = t[1:nsteps]

W = -4
noise = 0
input = 0
sigma = 0


model_params = Dict(:dt=>dt, :tau=>tau, :W=>[0 W; W 0], :nsteps=>nsteps, 
:noise=>noise, :input=>input, :sigma=>sigma, :const_add=>0, :init_add=>0)

forward = (startU; pars...) -> begin
    pars = Dict(pars)
    if haskey(pars, :W); 
        W=pars[:W];   # mess with it only if it is not already a matrix:
        if length(W)==1; pars=make_dict(["W"], [[0 W;W 0]], pars); end;
    end;     
    forwardModel(startU; pars...)
end



# clf();
# func = (;pars...) -> forward([-0.2, 0.3]; do_plot=true, merge(model_params, Dict(pars))...)
# func(;W=-4)

args = ["W", "const_add", ["start_add" 2], "sigma"]
params = [-4.01, 0.5, 0.2, -0.2, 0.01]

figure(1); clf();
value1, grad1, hess1 = keyword_vgh((;pars...)->forward([-0.2, 0.3]; do_plot=true, merge(model_params, Dict(pars))...)[1][1], args, params)
title(@sprintf("Running with dt=%g", dt))

dt = 0.005
t = 0:dt:1
tau = 0.1
nsteps = length(t)
t = t[1:nsteps]

model_params = Dict(:dt=>dt, :tau=>tau, :W=>[0 W; W 0], :nsteps=>nsteps, 
:noise=>noise, :input=>input, :sigma=>sigma, :const_add=>0, :init_add=>0)

figure(2); clf();
value2, grad2, hess2 = keyword_vgh((;pars...)->forward([-0.2, 0.3]; do_plot=true, fignum=2, merge(model_params, Dict(pars))...)[1][1], args, params)
title(@sprintf("Running with dt=%g", dt))

[grad1[:]' ; grad2[:]']

2×5 Array{Float64,2}:
 0.919879  -1.0215   1.0337    -1.04592   -0.270006
 0.915848  -1.00756  0.968136  -0.992003   0.571474

# TO-DOs


1. ===
2. ~~Optimize either an MGO or a ProAnti~~ DONE with MGO. Now on to ProAnti
3. ~~Set up so we can easily change task period durations in JJ as we run the model to evaluate the results of model-fitting~~ DONE
4. Have different task period durations while model-fitting
5. Set up to do searches over parameter space
6. Incorporate RT into fits?
10. If fluxSense is needed in ProAnti, could try choosing the Anti unit endpoint values by maximizing the |dJ/dw|^2 over those values.
11. Clean up the notebooks and write up what we've been doing!


# ProAnti

In [44]:
"""
    plot_PA(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, post_target_period=1,
        other_unused_params...)

Helper function for plotting ProAnti results
"""
function plot_PA(t, U, V; fignum=1, clearfig=true, rule_and_delay_period=1, target_period=1, post_target_period=1,
    other_unused_params...)
    figure(fignum)
    if clearfig; clf(); end
    
    ax1 = subplot(3,1,1)
    h = plot(t, V'); 
    setp(h[1], color=[0, 0, 1])
    setp(h[2], color=[1, 0, 0])
    setp(h[3], color=[1, 0.5, 0.5])
    setp(h[4], color=[0, 1, 1])
    ylabel("V")

    ax = gca()
    oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
    ylim(minimum([V[:];oldlims[1]])-0.1, maximum([V[:];oldlims[2]])+0.1)
    yl = [ylim()[1], ylim()[2]]
    vlines([rule_and_delay_period, 
            rule_and_delay_period+target_period,
            rule_and_delay_period+target_period+post_target_period], 
            -0.05, 1.05, linewidth=2)
    if yl[1]<0.02
        yl[1] = -0.02
    end
    if yl[2]>0.98
        yl[2] = 1.02
    end
    ylim(yl)
    grid(true)
    remove_xtick_labels(ax1)
        
    ax2 = subplot(3,1,2)
    hu = plot(t, U')
    oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
    ylim(minimum([U[:];oldlims[1]])-0.1, maximum([U[:];oldlims[2]])+0.1)
    setp(hu[1], color=[0, 0, 1])
    setp(hu[2], color=[1, 0, 0])
    setp(hu[3], color=[1, 0.5, 0.5])
    setp(hu[4], color=[0, 1, 1])
    ylabel("U"); 
    vlines([rule_and_delay_period, 
            rule_and_delay_period+target_period,
            rule_and_delay_period+target_period+post_target_period], 
            ylim()[1], ylim()[2], linewidth=2)
    remove_xtick_labels(ax2)

    grid(true)
    
    subplot(3,1,3)
    delta = V[1,:] - V[4,:]
    hr = plot(t, delta)
    oldlims = [ylim()[1]+0.1, ylim()[2]-0.1]
    ylim(minimum([delta[:];oldlims[1]])-0.1, maximum([delta[:];oldlims[2]])+0.1)
    vlines([rule_and_delay_period, 
            rule_and_delay_period+target_period,
            rule_and_delay_period+target_period+post_target_period], 
            ylim()[1], ylim()[2], linewidth=2)
    xlabel("t"); ylabel("Pro R - Pro L")
    grid(true)
        
end



plot_PA

In [89]:
model_params = Dict(
:dt     =>  0.02, 
:tau    =>  0.1, 
:vW     =>  -1.7,
:hW     =>  -1.7,
:sW     =>  0.2,
:dW     =>  0,
:nsteps =>  2, 
:noise  =>  [], 
:sigma  =>  0.08, 
:input  =>  0, 
:g_leak =>  0.25, 
:U_rest =>  -1,
:theta  =>  1, 
:beta   =>  1, 
:sw     =>  0.2,
:hw     =>  -1.7,
:vw     =>  -1.7,
:constant_excitation      => 0.19, 
:anti_rule_strength       => 0.1,
:pro_rule_strength        => 0.1, 
:target_period_excitation => 1,
:right_light_excitation   => 0.5, 
:right_light_pro_extra    => 0,
:const_add => 0, 
:init_add  => 0, 
:rule_and_delay_period    => 0.4,
:target_period            => 0.1,
:post_target_period       => 0.5,
:const_pro_bias           => 0,
)


function make_input(trial_type; dt=0.02, nderivs=0, difforder=0, constant_excitation=0.19, anti_rule_strength=0.1, 
    pro_rule_strength=0.1, target_period_excitation=1, right_light_excitation=0.5, right_light_pro_extra=0, 
    rule_and_delay_period=0.4, target_period=0.1, post_target_period=0.4, const_pro_bias=0,
    other_unused_params...)

    T = rule_and_delay_period + target_period + post_target_period
    t = 0:dt:T
    nsteps = length(t)

    input = constant_excitation + ForwardDiffZeros(4, nsteps, nderivs=nderivs, difforder=difforder)
    if trial_type=="Anti"
        input[2:3, t.<rule_and_delay_period] += anti_rule_strength
    elseif trial_type=="Pro"
        input[[1,4], t.<rule_and_delay_period] += pro_rule_strength
    else
        error("make_input: I don't recognize input type \"" * trial_type * "\"")
    end
    
    input[:,     (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += target_period_excitation
    input[1:2,   (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += right_light_excitation
    input[1,     (rule_and_delay_period.<=t) & (t.<rule_and_delay_period+target_period)] += right_light_pro_extra
    
    input[[1,4],:] += const_pro_bias
    
    return input, t, nsteps
end


function run_ntrials(nPro, nAnti; plot_list=[], nderivs=0, difforder=0, model_params...)
    pro_input,  t, nsteps = make_input("Pro" ; model_params...)
    anti_input, t, nsteps = make_input("Anti"; model_params...)

    model_params = Dict(model_params)
    sW = model_params[:sW]
    hW = model_params[:hW]
    vW = model_params[:vW]
    dW = model_params[:dW]
    model_params = make_dict(["nsteps", "W"], [nsteps, [sW vW dW hW; vW sW hW dW; dW hW sW vW; hW dW vW sW]], 
        model_params)
    model_params = make_dict(["nderivs", "difforder"], [nderivs, difforder], model_params)
    
    proVs  = ForwardDiffZeros(4, nPro, nderivs=nderivs, difforder=difforder)
    antiVs = ForwardDiffZeros(4, nAnti, nderivs=nderivs, difforder=difforder)

    # --- PRO ---
    if length(plot_list)>0; figure(1); clf(); end
    model_params = make_dict(["input"], [pro_input], model_params)
    for i=1:nPro
        startU = [-0.3, -0.7, -0.7, -0.3]
        Uend, Vend, U, V = forwardModel(startU, do_plot=false; model_params...)
        proVs[:,i] = Vend
        if any(plot_list.==i) 
            plot_PA(t, U, V; fignum=1, clearfig=false, model_params...)
            subplot(3,1,1); title("PRO")
        end
    end

    # --- ANTI ---
    if length(plot_list)>0; figure(2); clf(); end
    model_params = make_dict(["input"], [anti_input], model_params)
    for i=1:nAnti
        startU = [-0.7, -0.3, -0.3, -0.7]
        Uend, Vend, U, V = forwardModel(startU, do_plot=false; model_params...)
        antiVs[:,i] = Vend
        if any(plot_list.==i) 
            plot_PA(t, U, V; fignum=2, clearfig=false, model_params...)
            subplot(3,1,1); title("ANTI")
        end
    end
    
    return proVs, antiVs
end

nPro = 10; nAnti = 5;
proVs, antiVs = @time(run_ntrials(nPro, nAnti; plot_list=[1:5;], model_params...))

@printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/nPro)
@printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/nAnti)


  1.346095 seconds (167.06 k allocations: 7.927 MB, 1.17% gc time)
Pro % correct = 100%
Anti % correct = 

80% 


In [17]:
function JJ(nPro, nAnti; pro_target=0.9, anti_target=0.7, 
    theta1=0.025, theta2=0.035, cbeta=0.003, verbose=false, 
    pre_string="", zero_last_sigmas=0, seedrand=NaN, 
    rule_and_delay_periods = [0.4], target_periods = [0.1], post_target_periods = [0.5],
    nderivs=0, difforder=0, model_params...)

    nruns = length(rule_and_delay_periods)*length(target_periods)*length(post_target_periods)
    
    cost1s = ForwardDiffZeros(1, nruns, nderivs=nderivs, difforder=difforder)
    cost2s = ForwardDiffZeros(1, nruns, nderivs=nderivs, difforder=difforder)

    if ~isnan(seedrand); srand(seedrand); end
    
    n = totHitsP = totHitsA = totDiffsP = totDiffsA = 0
    for i in rule_and_delay_periods
        for j in target_periods
            for k = post_target_periods
                n += 1
                
                my_params = make_dict(["rule_and_delay_period", "target_period", "post_target_period"],
                [i, j, k], Dict(model_params))
    
                # print("model params is " ); print(model_params); print("\n")
                proVs, antiVs = run_ntrials(nPro, nAnti; nderivs=nderivs, difforder=difforder, my_params...)

                hitsP  = 0.5*(1 + tanh.((proVs[1,:]-proVs[4,:,])/theta1))
                diffsP = tanh.((proVs[1,:,]-proVs[4,:])/theta2).^2
                hitsA  = 0.5*(1 + tanh.((antiVs[4,:]-antiVs[1,:,])/theta1))
                diffsA = tanh.((antiVs[4,:,]-antiVs[1,:])/theta2).^2

                if nPro>0 && nAnti>0
                    cost1s[n] = (nPro*(mean(hitsP) - pro_target).^2  + nAnti*(mean(hitsA) - anti_target).^2)/(nPro+nAnti)
                    cost2s[n] = -cbeta*(nPro*mean(diffsP) + nAnti*mean(diffsA))/(nPro+nAnti)
                elseif nPro>0
                    cost1s[n] = (mean(hitsP) - pro_target).^2
                    cost2s[n] = -cbeta*mean(diffsP)
                else
                    cost1s[n] = (mean(hitsA) - anti_target).^2
                    cost2s[n] = -cbeta*mean(diffsA)
                end

                totHitsP  += mean(hitsP);  totHitsA  += mean(hitsA); 
                totDiffsP += mean(diffsP); totDiffsA += mean(diffsA);
            end
        end
    end
    
    cost1 = mean(cost1s)
    cost2 = mean(cost2s)

    hitsP = totHitsP/n; hitsA = totHitsA/n; diffsP = totDiffsP/n; diffsA = totDiffsA/n
    
    if verbose
        @printf("%s", pre_string)
        @printf("     -- cost=%g,   cost1=%g, cost2=%g\n", 
            convert(Float64, cost1+cost2), convert(Float64, cost1), convert(Float64, cost2))
        if nPro>0 && nAnti>0
            @printf("     -- mean(hitsP)=%g, mean(diffsP)=%g mean(hitsA)=%g, mean(diffsA)=%g\n", 
                convert(Float64, mean(hitsP)), convert(Float64, mean(diffsP)),
                convert(Float64, mean(hitsA)), convert(Float64, mean(diffsA)))
        elseif nPro>0
            @printf("     -- mean(hitsP)=%g, mean(diffsP)=%g (nAnti=0)\n", 
                convert(Float64, mean(hitsP)), convert(Float64, mean(diffsP)))
        else
            @printf("     -- (nPro=0) mean(hitsA)=%g, mean(diffsA)=%g\n", 
                convert(Float64, mean(hitsA)), convert(Float64, mean(diffsA)))
        end        
    end
    
    return cost1 + cost2
end



JJ(2, 10; plot_list=1:5, verbose=true, model_params...)


     -- cost=-0.000412731,   cost1=0.000445974, cost2=-0.000858706
     -- mean(hitsP)=0.854628, mean(diffsP)=0.314139 mean(hitsA)=0.68889, mean(diffsA)=0.280654


-0.0004127313597031488

### Example of taking the gradient and Hessian of the cost function JJ()

In [20]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias"]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]

func = (;params...) -> JJ(100, 0; rule_and_delay_periods = [0.4, 0.8], seedrand=30, cbeta=0.01, 
plot_list = 1:5, verbose=false, merge(model_params, Dict(params))...)

cost, grad, hess = keyword_vgh(func, args, seed)

# func(;make_dict(args, seed+ [1,0.2,0,0,0,0,0])...) - func(;make_dict(args, seed)...)


(0.07535583378383748,[0.0570604,-0.00251589,-0.0368942,0.288332,-0.48954,0.0181813,0.00435835],
[0.283457 -0.086324 … 0.00977107 1.02778; -0.086324 0.00153948 … -0.000470085 -0.191414; … ; 0.00977107 -0.000470085 … 0.00702631 0.108539; 1.02778 -0.191414 … 0.108539 3.61222])

# Sandbox

A place to play around

In [342]:
bbox

Dict{Symbol,Any} with 9 entries:
  :constant_excitation      => [-2 2]
  :vW                       => [-3 3]
  :target_period_excitation => [0.05 4.0]
  :const_pro_bias           => [-2 2]
  :right_light_excitation   => [0.05 4.0]
  :hW                       => [-3 3]
  :dW                       => [-3 3]
  :sigma                    => [0.01 0.2]
  :sW                       => [0 3]

In [ ]:

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   1,   0.2,  1,    0.39,                0.15,                       0.1]

args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]


# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2])

model_params = merge(model_params, Dict(:post_target_period=>0.5))
# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]


# ======== SEARCH ZONE:

sbox = Dict(:sW=>[0.001 0.5], :vW=>[-0.5 0.5], :hW=>[-0.5 0.5], :dW=>[-0.5 0.5],
:constant_excitation=>[-0.5 0.5], :right_light_excitation=>[0.1 0.5], :target_period_excitation=>[0.1 0.5],
:const_pro_bias=>[0 0.2], :sigma=>[0.02 0.19])

cbetas = [0.002, 0.01]

basename = "farm_TEST_"

while true
    myseed = seed;
    sr = convert(Int64, round(time()))

    myseed = copy(seed);
    for i=1:length(args)
        sym = Symbol(args[i])
        if haskey(sbox, sym)
            myseed[i] = sbox[sym][1] + diff(sbox[sym],2)[1]*rand()
        end
    end
    nPro=100; nAnti=100

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    for cb in cbetas
        @printf("Going with seed = "); print_vector_g(myseed); print("\n")
        pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(myseed, args, bbox, 
            (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            post_target_periods=post_target_periods,
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...),
            start_eta = 0.01, tol=1e-12, verbose=true, maxiter=2)
        @printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

        matwrite(next_file(basename, 4), Dict("args"=>args, "myseed"=>myseed, "pars"=>pars, "traj"=>traj,
        "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>nPro, "nAnti"=>nAnti, "sr"=>sr, "cb"=>cb, 
        "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox)))
    end
end


In [313]:
"""
cost = standard_cost(args, pars, sr, model_params)

Computes the cost as if cb had been 0.01
"""
function standard_cost(args, pars, sr, model_params)
    cb = 0.01
    nPro=100; nAnti=100

    rule_and_delay_periods = [0.4, 1.2]
    post_target_periods    = [0.5, 1.5]

    func = (;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
            post_target_periods=post_target_periods,
            seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)
    
    return func(;make_dict(args, pars)...)
end
    
"""
cost = standard_cost(filename)

Returns the standard cost (at cb=0.01) and inserts it into the file with key "scost" if it wasn't there
already
"""
function standard_cost(filename; verbose=false)
    A = matread(filename)
    if !haskey(A, "scost")
        get!(A, "scost", standard_cost(A["args"], A["pars"], A["sr"], symbol_key_ize(A["model_params"])))
        if verbose
            @printf("File %s did not have scost, adding its value %g\n", filename, A["scost"])
        end
        matwrite(filename, A)
    end
    return A["scost"]
end

standard_cost

In [314]:
standard_cost("farm_C_0104")

-0.007350536801978829

Dict{String,Any} with 14 entries:
  "traj"         => [0.01 0.011 … 0.0240117 0.0264129; 0.0916161 0.0906175 … -0…
  "scost"        => -0.00634099
  "cpm_traj"     => [56.0 5.0 … 101.0 97.0; 0.0 0.0 … 0.0 0.0]
  "model_params" => Dict{String,Any}(Pair{String,Any}("const_pro_bias",0),Pair{…
  "sr"           => 1505150372
  "args"         => Any["sW","vW","hW","dW","constant_excitation","right_light_…
  "nAnti"        => 100
  "nPro"         => 100
  "bbox"         => Dict{String,Any}(Pair{String,Any}("sigma",[0.01 0.2]),Pair{…
  "cost"         => -0.00118598
  "myseed"       => [0.287614,0.0213833,-0.464603,0.305447,-0.0605855,0.470735,…
  "pars"         => [0.0342661,-0.927829,0.70743,1.32046,0.0516115,0.441745,0.4…
  "cb"           => 0.002
  "sbox"         => Dict{String,Any}(Pair{String,Any}("sigma",[0.02 0.19]),Pair…

In [295]:
A = matread("farm_A_0001")
@printf("At cb=0.002, cost was %g. At cb=0.01, cost was %g\n", A["cost"], standard_cost("farm_A_0001"))


At cb=0.002, cost was -0.00118598. At cb=0.01, cost was -0.00634099


In [379]:
A = matread("farm_E_0005")
model_params = symbol_key_ize(A["model_params"])
args         = A["args"];
nAnti        = A["nAnti"]
nPro         = A["nPro"]
pars         = A["pars"]
sr           = A["sr"]
traj         = A["traj"]
@printf("Cost went from %g to %g ; max iters on cpm was %d\n\n", traj[2,1], traj[2,end], maximum(cpm_traj[1,:]))
A

Cost went from 0.0846739 to -0.0160859 ; max iters on cpm was 13



Dict{String,Any} with 18 entries:
  "traj"                   => [0.01 0.011 … 9.19729e-6 4.59865e-6; 0.0846739 0.…
  "scost"                  => -0.00853743
  "cpm_traj"               => [14.0 10.0 … 89.0 83.0; 0.0 0.0 … 0.0 0.0]
  "post_target_periods"    => [0.5,1.5]
  "rule_and_delay_periods" => [0.4,1.2]
  "model_params"           => Dict{String,Any}(Pair{String,Any}("const_pro_bias…
  "sr"                     => 1505318661
  "args"                   => Any["sW","vW","hW","dW","constant_excitation","ri…
  "nAnti"                  => 100
  "theta2"                 => 0.25
  "nPro"                   => 100
  "theta1"                 => 0.15
  "bbox"                   => Dict{String,Any}(Pair{String,Any}("sigma",[0.01 0…
  "cost"                   => -0.0160859
  "myseed"                 => [0.253053,-0.476459,-0.0127227,-0.322372,0.015171…
  "pars"                   => [0.121312,-2.26178,-0.222131,0.472211,0.401695,3.…
  "cb"                     => 0.02
  "sbox"                   => 

In [381]:
good_seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]

[args pars good_seed]

9×3 Array{Any,2}:
 "sW"                         0.121312    0.0840597
 "vW"                        -2.26178    -1.32677  
 "hW"                        -0.222131   -0.437334 
 "dW"                         0.472211   -0.324835 
 "constant_excitation"        0.401695    0.567997 
 "right_light_excitation"     3.99987     0.712216 
 "target_period_excitation"   0.0501671   0.0500075
 "const_pro_bias"             0.0382744   0.0858569
 "sigma"                      0.2         0.25     

In [386]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 0.2; 0.4; 0.02], model_params)

run_factor = 10

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; verbose=true, seedrand=sr, rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


  4.884860 seconds (4.66 M allocations: 286.025 MB, 1.52% gc time)
Pro % correct = 95%
Anti % correct = 70.9% 
     -- cost=-0.000401259,   cost1=0.0023092, cost2=-0.00271046
     -- mean(hitsP)=0.965516, mean(diffsP)=0.984829 mean(hitsA)=0.681942, mean(diffsA)=0.822143


-0.00040125882805575096

### Evaluating the results of the farm

In [378]:
basename = "farm_E"
fnames = readdir()
matched_filenames = Array{Bool}(length(fnames))
for i=1:length(fnames)
    matched_filenames[i] = ismatch(Regex(@sprintf("^%s", basename)), fnames[i])
end
myguys = fnames[find(matched_filenames)]
costs = zeros(size(myguys))
cbs   = zeros(size(myguys))
for i=1:length(myguys)
    A = matread(myguys[i])
    costs[i] = standard_cost(myguys[i]; verbose=true)
    cbs[i]   = A["cb"]
end

cbset = unique(cbs)

figure(3); clf();
for i=1:length(cbset)    
    ax1 = subplot(length(cbset),1,i)
    h1 = plt[:hist](costs[find(cbs.==cbset[i])], -0.02:0.001:0.008);
    ylabel("# of runs")
    title(@sprintf("cb=%g  (%d total)", cbset[i], length(find(cbs.==cbset[i]))))
    if i < length(cbset)
        remove_xtick_labels(ax1)
    end
    if i==length(cbset)
        xlabel("final cost")
    end
end
        
unique(cbs)

2-element Array{Float64,1}:
 0.02
 0.04

In [376]:
myguys[find(costs.<=-0.0075)]

21-element Array{String,1}:
 "farm_E_0005"
 "farm_E_0012"
 "farm_E_0018"
 "farm_E_0027"
 "farm_E_0031"
 "farm_E_0036"
 "farm_E_0038"
 "farm_E_0039"
 "farm_E_0041"
 "farm_E_0043"
 "farm_E_0047"
 "farm_E_0049"
 "farm_E_0053"
 "farm_E_0054"
 "farm_E_0065"
 "farm_E_0067"
 "farm_E_0086"
 "farm_E_0110"
 "farm_E_0126"
 "farm_E_0135"
 "farm_E_0136"

# Example of 9-param, including diagonal weight and sigma, and robust across 3x in rule/delay period and post-target period

In [37]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "sigma"]
seed = [0.2,   -1.7, -1.7,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   0.17,  0.17,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      -0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                     0.1]
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :sigma=>[0.05 1])
model_params = merge(model_params, Dict(:post_target_period=>0.5))


# Now with constant_pro_bias and a fixed sigma=0.1
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.2,   -1.7, -1.7,  0,    0.39,                0.15,                       0.1]
args = [args ; ["const_pro_bias", "sigma"]]
seed = [seed ; [0.1,               0.1]]
model_params = merge(model_params, Dict(:post_target_period=>0.5))
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.25])

# seed = [0.0840597,  -1.32677,  -0.437334,  -0.324835,  0.567997, 0.712216,  0.0500075,  0.0858569,  0.25]

# ==========

nPro=100; nAnti=100

rule_and_delay_periods = [0.4, 1.2]
post_target_periods    = [0.5, 1.5]

pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(seed, args, bbox, 
(;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
post_target_periods=post_target_periods,
seedrand=31, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...),
start_eta = 0.01, tol=1e-9, verbose=true)

pars'

0: eta=0.01 ps=[0.200, -1.700, -1.700, 0.000, 0.390, 0.150, 0.100, 0.100, 0.100]
     -- cost=0.0569741,   cost1=0.0664577, cost2=-0.00948358
     -- mean(hitsP)=0.585719, mean(diffsP)=0.964404 mean(hitsA)=0.521092, mean(diffsA)=0.932312
     -- cost=0.0546503,   cost1=0.0640934, cost2=-0.00944312
     -- mean(hitsP)=0.59576, mean(diffsP)=0.960612 mean(hitsA)=0.518412, mean(diffsA)=0.928012
1: eta=0.011 cost=0.0546503 jtype=constrained costheta=-0.929 ps=[0.200008, -1.70008, -1.69985, -0.000825069, 0.388531, 0.150065, 0.0999995, 0.102269, 0.0912348]
     -- cost=0.0517394,   cost1=0.0611855, cost2=-0.00944606
     -- mean(hitsP)=0.606351, mean(diffsP)=0.958814 mean(hitsA)=0.51702, mean(diffsA)=0.930398
2: eta=0.0121 cost=0.0517394 jtype=constrained costheta=-0.590 ps=[0.200133, -1.70019, -1.70003, -0.00241266, 0.388902, 0.150087, 0.0999993, 0.112622, 0.0884086]
     -- cost=0.049832,   cost1=0.059234, cost2=-0.00940196
     -- mean(hitsP)=0.617304, mean(diffsP)=0.959881 mean(hitsA)=0.5

1×9 Array{Float64,2}:
 0.0840597  -1.32677  -0.437334  -0.324835  …  0.0500075  0.0858569  0.25

In [47]:
[args pars]

9×2 Array{Any,2}:
 "sW"                         0.0840597
 "vW"                        -1.32677  
 "hW"                        -0.437334 
 "dW"                        -0.324835 
 "constant_excitation"        0.567997 
 "right_light_excitation"     0.712216 
 "target_period_excitation"   0.0500075
 "const_pro_bias"             0.0858569
 "sigma"                      0.25     

In [53]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 1.5; 1; 0.01], model_params)

run_factor = 10

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


 17.340804 seconds (33.19 M allocations: 1.907 GB, 2.10% gc time)
Pro % correct = 87.1%
Anti % correct = 67.2% 


-0.00115953051519695

# Example of a ProAnti network optimization

This one uses two rule periods and two post_target_periods to try to get some stability in the trained network.

In [27]:
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "sigma"]
seed = [0.2,   -1.7, -1.7,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   0.17,  0.17,      0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      -0.19,                0.5,                       1,                       0.1]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :sigma=>[0.05 1])
model_params = merge(model_params, Dict(:post_target_period=>0.5))


# Now with constant_pro_bias and a fixed sigma=0.1
args = ["sW", "vW", "hW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias"]
seed = [0.2,   -1.7, -1.7,      0.39,                0.15,                       0.1,                       0.1]
model_params = merge(model_params, Dict(:post_target_period=>0.5, :sigma=>0.1))
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0.05 4], :const_pro_bias=>[-2 2])

# ==========

nPro=100; nAnti=100

rule_and_delay_periods = [0.4, 0.8]
post_target_periods    = [0.5, 1]

pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(seed, args, bbox, 
(;params...) -> JJ(nPro, nAnti; rule_and_delay_periods=rule_and_delay_periods,
post_target_periods=post_target_periods,
seedrand=31, cbeta=0.01, verbose=true, merge(model_params, Dict(params))...),
start_eta = 0.01, tol=1e-12, verbose=true, maxiter=2)

pars'

0: eta=0.01 ps=[0.200, -1.700, -1.700, 0.390, 0.150, 0.100, 0.100]
     -- cost=0.0609354,   cost1=0.0702293, cost2=-0.0092939
     -- mean(hitsP)=0.574693, mean(diffsP)=0.952638 mean(hitsA)=0.522901, mean(diffsA)=0.906143
     -- cost=0.0594435,   cost1=0.0686243, cost2=-0.00918085
     -- mean(hitsP)=0.57821, mean(diffsP)=0.941545 mean(hitsA)=0.525837, mean(diffsA)=0.894625
1: eta=0.011 cost=0.0594435 jtype=constrained costheta=-0.985 ps=[0.199944, -1.70014, -1.69931, 0.382009, 0.150148, 0.0999991, 0.0947458]
     -- cost=0.0586714,   cost1=0.067794, cost2=-0.0091226
     -- mean(hitsP)=0.583192, mean(diffsP)=0.934959 mean(hitsA)=0.522194, mean(diffsA)=0.889561
2: eta=0.0121 cost=0.0586714 jtype=constrained costheta=-0.428 ps=[0.200019, -1.70054, -1.69829, 0.373021, 0.150443, 0.0999975, 0.0994822]


1×7 Array{Float64,2}:
 0.200019  -1.70054  -1.69829  0.373021  0.150443  0.0999975  0.0994822

In [29]:
cost

0.05867137476232549

In [312]:
# --------------------
# NOW EVALUATE RESULTS HERE
# --------------------

my_params = make_dict([args; "plot_list"; "post_target_period"; "rule_and_delay_period"; "dt"], 
[pars; [[1:10;]]; 1.5; 0.5; 0.02], model_params)

run_factor = 10

proVs, antiVs = @time(run_ntrials(nPro*run_factor, nAnti*run_factor; plot_list=[], my_params...))

if nPro>0;  @printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/(nPro*run_factor)); end;
if nAnti>0; @printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/(nAnti*run_factor)); end;


figure(3); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-1:0.02:1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-1:0.02:1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])

figure(1); clf(); figure(2); clf();

JJ(nPro, nAnti; rule_and_delay_periods=my_params[:rule_and_delay_period], 
post_target_periods=my_params[:post_target_period], my_params...)


  8.740874 seconds (13.19 M allocations: 798.834 MB, 1.98% gc time)
Pro % correct = 90.7%
Anti % correct = 67.1% 


-0.0016377281161004287

In [138]:
######################################################
#                                                    #
#         BBOX_HESSIAN_KEYWORD_MINIMIZATION          #
#                                                    #
######################################################




"""
pdict = wallwrap(bdict, pdict)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, and pdict, a dictionary of symbols
to values (or, alternatively, an Array of (Symbol, value) tuples], goes through each of the symbols in 
bdict and modifies the corresponding value in pdict putting it through a tanh so the final output lies 
within the limits in bdict.  Returns the new pdict.  Makes a copy of pdict so as not to modify the original.
"""
function wallwrap(bdict, epdict)
    local pdict = two_level_copy(epdict)
    if typeof(pdict)<:Array
        pdict = Dict(pdict)
    end

    allkeys = keys(bdict)

    for k in allkeys
        local bbox = bdict[k]
        d = 0.5*(bbox[2] - bbox[1])
        m = 0.5*(bbox[2] + bbox[1])

        pdict[k] = bbox[1] + d*(tanh((pdict[k]-m)/d)+1)
    end
    return pdict
end

    
"""
params = vector_wrap(bbox, args, eparams)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, args, an array of strings representing
symbols, and params, an array of values corresponding to the args list, puts each param that has an entry 
in bdict through the tanh-walling mechanism, and returns the result. Does not modify the contents of the 
original params vector (or bdict or args).
"""
function vector_wrap(bbox, args, eparams)
    local params = two_level_copy(eparams)
    pdict = wallwrap(bbox, make_dict(args, params))
    i=1; j=1
    for i=1:length(args)
        if typeof(args[i])<:Array
            params[j:j+args[i][2]-1] = pdict[Symbol(args[i][1])]
            j += args[i][2]-1
        else
            params[j] = pdict[Symbol(args[i])]
        end
    j = j+1
    end
    return params
end


"""
params = inverse_wall(bdict, args, wparams)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, args, an array of strings representing
symbols, and wparams, an array of values corresponding to the args list where each param that has an entry 
in bdict has alreadt been through the tanh-walling mechanism, UNwalls the ones that have a bdict entry and
returns the result. Does not modify the contents of the original params vector (or bdict or args).
"""
function inverse_wall(bdict, args, wparams)
    local params = two_level_copy(wparams)
    pdict = inverse_wall(bdict, make_dict(args, params))
    i=1; j=1
    for i=1:length(args)
        if typeof(args[i])<:Array
            params[j:j+args[i][2]-1] = pdict[Symbol(args[i][1])]
            j += args[i][2]-1
        else
            params[j] = pdict[Symbol(args[i])]
        end
        j = j+1
    end
    return params    
end

    
"""
pdict = inverse_wall(bdict, wdict)
Given bdict, a dictionary of symbols to [minval, maxval] vectors, and wdict, a dictionary of symbols to values
(or vectors of values)  UNwalls the ones that have a bdict entry and
returns the result. Does not modify the contents of any dictionaries.
"""
function inverse_wall(bdict, wdict)
    local pdict = two_level_copy(wdict)

    allkeys = keys(bdict)
    for k in allkeys
        local bbox = bdict[k]
        d = 0.5*(bbox[2] - bbox[1])
        m = 0.5*(bbox[2] + bbox[1])

        try
            pdict[k] = m + d*0.5*log((pdict[k]-bbox[1])./(2*d - pdict[k] + bbox[1]))
        catch
            error(@sprintf("Had trouble with key %s", string(k)))
        end
    end
    return(pdict)
end
  


"""
function bbox_Hessian_keyword_minimization(seed, args, bbox, func; wallwidth=NaN, start_eta=10, tol=1e-6, 
    maxiter=400, verbose=false)

Like constrained_Hessian_minimization, but uses keyword_hessian!(). 

# PARAMETERS:

- seed        column vector, representing the starting value of the parameters.

- args        List of strings identifying parameters for differentiation, e.g., ["const_E", "w_self]

- bbox        If softbox=true (the default), should then be a Dict of Symbol=>[minval maxval] entries. An entry
            in this Dict indicates that the corresponding parameter is to be bounded, as indicated by the associated 
            [minval maxval] vector. The bbox dictionary can have fewer entries than the number of parameters, and its
            default value is Dict(), indicating an unbounded search.
                If softbox=false, then bbox should be an nargs-by-2 matrix indicating the range for each argument,
            with the minima (first column) and maxima (second column), and entries for ALL parameters.

- func        func must take only optional keyword args, and must 
            take nderivs=0, difforder=0  and declare any new matrices using ForwardDiffZeros() instead of zeros()


# OPTIONAL PARAMETERS:

- start_eta    Starting value of the radius.  It's good to start with somethibg biggish, if it is
             too much, it'll quickly get cut down.

- tol=1e-6     Numerical tolerance. If a proposed jump produces a change in func that is less than
             this, the minimization stops.

- maxiter=400  Maximum number of iterations to do before stopping

- verbose=false   If true, print out a report on each iteration of iteration number, radius size (eta),
                what type jump was proposed ("Newton" means going straight to global min, "constrained" means jump has 
                norm eta, failed means that finding the minimum at a given radius somehow didn't work). Will also
                print out the cosine of the angle between the proposed jump and the gradient.

- verbose_level   If less than 2, regular verbose output, if 2 or greater, very verbose, for debugging.

- softbox         If true, then bbox must be a Dict() and we use the tanh() mechanism for putting a fixed limit
                on the parameters.

- hardbox=false   If true, ignores wallwidth, and just rests parameter values to the bounding box if they go outside it.
                If false, adds cost function "walls" to implement the bounding box.

- walldith=NaN     Used for putting up cost function "walls" that implement the bounding box limits. Can be NaN.
                If it is NaN, then the wallwidth is a constant factor of the range width for each argument. If not NaN, must
                be an nargs-long vector that indicates the actual wall widths.

- wallwidth_factor=0.18   Only relevant if wallwidth is NaN, otherwise ignored. For each arg, the wall width
                is going to be wall_width_factor*(bbox[i,2] - bbox[i,1])


# RETURNS:

- params       A vector the size of seed that has the last values of the minimizing parameters for func
- trajectory   A (2+length(params))-by-nsteps matrix. Each column corresponds to an iteration step, and contains
                 the value of eta used, the cost, and the value of the parameters at that iteration
- cost         Final value of objective function
- cpm_traj     A 2-by-nsteps matrix, containing reports from the contrained parabolic minimization at each timestep.
             The first row is niters (how many iterations cpm's 1-d minimization ran for) and the second row is
             Dlambda, the last change in the parameter being minimized in cpm's internal search


# EXAMPLE:

```
function tester(;x=5, y=10, z=20, nderivs=0, difforder=0)
    return x^2*y + z/tanh(y)
end

params, trajectory = bbox_Hessian_keyword_minimization([0.5, 0.5], ["x", "y"], [1.1 2 ; 1.1 4], tester, 
    verbose=true, tol=1e-12, start_eta=1);
```


"""
function bbox_Hessian_keyword_minimization(seed, args, bbox, func; start_eta=0.1, tol=1e-6, maxiter=400,
    verbose=false, verbose_level=1, verbose_every=1, 
    softbox=true, hardbox=false, wallwidth=NaN, wallwidth_factor=0.18)

      
    """
    Given args, a list of string representing the arguments of interest, a bounding box for each,
    and a Symbol=>value dictionary with the corresponding parameters, computes and returns a high cost for 
    being outside the bounding box
    """
    function wall_cost(args, bbox; wallwidth=NaN, nderivs=0, difforder=0, pars...) 
        myparams = ForwardDiffZeros(length(pars), 1, nderivs=nderivs, difforder=difforder)
        pars2 = Dict()
        for i in [1:length(pars);]
            pars2[string(pars[i][1])] = pars[i][2]
        end
        for i in [1:length(args);]
            myparams[i] = pars2[args[i]]
        end
        
        if isnan(wallwidth)
            # We know that we're going to be taking hessian for params, so declare zeros accordingly:
            wallwidth = ForwardDiffZeros(length(myparams), 1, nderivs=nderivs, difforder=difforder)

            for i in [1:length(myparams);]
                wallwidth[i] = wallwidth_factor*(bbox[i,2]-bbox[i,1])
            end
        end

        retval = 0
        for i in [1:length(myparams);]
            if myparams[i]<bbox[i,1]
                retval += cosh((bbox[i,1]-myparams[i])/wallwidth[i])-1.0
            elseif bbox[i,2] < myparams[i]
                retval += cosh((myparams[i]-bbox[i,2])/wallwidth[i])-1.0                
            end
        end

        return 2*retval
    end

    traj_increment = 100
    params = 0  # Make sure to have this here so that params stays defined beyond the try/catch
    if ( !(typeof(bbox)<:Dict) ); error("Currently only supporting softbox=true, bbox must be a Dict"); end;
    try
        params = copy(inverse_wall(bbox, args, seed))
    catch y
        @printf("inverse_wall failed with error %s\n", y)
        error("Were all initial param values within the indicated walls?")
    end
    eta = start_eta
    trajectory = zeros(2+length(params), traj_increment); cpm_traj = zeros(2, traj_increment)

    if verbose
        @printf "%d: eta=%g ps=" 0 eta 
        print_vector(vector_wrap(bbox, args, params))
        @printf "\n"
    end
    
    if softbox
        if !(typeof(bbox)<:Dict); error("bhm: If softbox=true, then bbox must eb a Dict"); end
        cost, grad, hess = keyword_vgh((;pars...)->func(;wallwrap(bbox, pars)...), args, params)
    elseif hardbox
        cost, grad, hess = keyword_vgh((;pars...) -> func(;pars...), args, params)
    else
        cost, grad, hess = keyword_vgh((;pars...) -> func(;pars...) + wall_cost(args, bbox; wallwidth=wallwidth, pars...),
            args, params)        
    end
        
    chessdelta = zeros(size(params))
    
    i=0  # here so variable i is available outside the loop
    for i in [1:maxiter;]
        if i > size(trajectory, 2)
            trajectory = [trajectory zeros(2+length(params), traj_increment)]
            cpm_traj   = [cpm_traj   zeros(2, traj_increment)]
        end
        trajectory[1:2, i]   = [eta;cost]
        trajectory[3:end, i] = vector_wrap(bbox, args, params)
        
        hessdelta  = - inv(hess)*grad
        try
            if verbose && verbose_level >= 2
                @printf("bhm: about to try cpm with grad : "); print_vector_g(grad); print("\n")
                @printf("bhm:   hess :"); print_vector_g(hess[:]); print("\n");
            end
            if verbose && verbose_level >= 2
                cpm_out = constrained_parabolic_minimization(hess, grad'', eta, 
                    maxiter=500, tol=1e-20, do_plot=true, verbose=true)                
            else
                cpm_out = constrained_parabolic_minimization(hess, grad'', eta, maxiter=500, tol=1e-20)
            end
            chessdelta = cpm_out[1]; cpm_traj[1,i] = cpm_out[5]; cpm_traj[2,i] = cpm_out[6]
            jumptype = "not failed"
        catch y
            jumptype = "failed"
            if verbose
                @printf "Constrained parabolic minimization failed with error %s\n" y
                @printf "\n"
                @printf "eta was %g\n" eta
                @printf "grad was\n"
                print_vector(grad)
                @printf "\n\nhess was\n"
                for k in [1:length(grad);]
                    print_vector(hess[k,:])
                    @printf "\n"
                end
                @printf "\n"
                matwrite("error_report.mat", Dict("grad"=>grad, "hess"=>hess, "eta"=>eta))
            end
            break
        end

        if norm(hessdelta) <= eta
            new_params = params + hessdelta
            jumptype = "Newton"
        elseif jumptype != "failed" 
            new_params = params + chessdelta
            jumptype  = "constrained"
        end

        if jumptype != "failed"
            if softbox
                new_cost, new_grad, new_hess = 
                    keyword_vgh((;pars...) -> func(;wallwrap(bbox, pars)...), args, new_params)
                if verbose && verbose_level >=2
                    @printf("bhm: had new_params = : "); print_vector_g(vector_wrap(bbox, args, params)); print("\n");
                    @printf("bhm: and my bbox was : "); print(bbox); print("\n")
                    @printf("bhm: and my wallwrap output was : "); print(wallwrap(bbox, make_dict(args, new_params))); print("\n")
                    @printf("bhm: and this produced new_grad : "); print_vector_g(new_grad); print("\n")
                    @printf("bhm:   new_hess :"); print_vector_g(new_hess[:]); print("\n");                                        
                end
            elseif hardbox
                for p in [1:length(new_params);]
                    if new_params[p] < bbox[p,1]; new_params[p] = bbox[p,1]; end
                    if bbox[p,2] < new_params[p]; new_params[p] = bbox[p,2]; end
                 end        
                
                new_cost, new_grad, new_hess = keyword_vgh((;pars...) -> func(;pars...), args, new_params)
            else
                new_cost, new_grad, new_hess = keyword_vgh((;pars...) -> func(;pars...) + 
                        wall_cost(args, bbox; wallwidth=wallwidth, pars...),
                    args, new_params)                
            end
            
            if abs(new_cost - cost) < tol || eta < tol
                if verbose
                    @printf("About to break -- tol=%g, new_cost-cost=%g, eta=%g\n", tol, new_cost-cost, eta)
                end
                break
            end
        end

        if jumptype == "failed" || new_cost >= cost  
            if verbose
                @printf("eta going down: new_cost-cost=%g and jumptype='%s'\n", new_cost-cost, jumptype)
                if verbose_level >= 2
                    nwp = vector_wrap(bbox, args, new_params); wp = vector_wrap(bbox, args, params)
                    @printf("   vvv: proposed new params were : "); print_vector_g(nwp); print("\n")
                    @printf("   vvv: proposed delta params was : "); print_vector_g(nwp-wp); print("\n")
                    @printf("   vvv: grad was : "); print_vector_g(grad); print("\n")
                    costheta = dot(new_params-params, grad)/(norm(new_params-params)*norm(grad))
                    @printf("   vvv: costheta of proposed jump was %g\n", costheta)
                end
            end
            eta = eta/2
            costheta = NaN
            if eta < tol
                if verbose
                    @printf("About to break -- tol=%g, new_cost-cost=%g, eta=%g\n", tol, new_cost-cost, eta)
                end
                break
            end
        else
            eta = eta*1.1
            costheta = dot(new_params-params, grad)/(norm(new_params-params)*norm(grad))

            params = new_params
            cost = new_cost
            grad = new_grad
            hess = new_hess
        end

        if verbose
            if rem(i, verbose_every)==0
                @printf "%d: eta=%g cost=%g jtype=%s costheta=%.3f ps=" i eta cost jumptype costheta
                print_vector_g(vector_wrap(bbox, args, params))
                @printf "\n"
                if verbose_level >= 3
                    @printf "    At this point, grad is ="
                    print_vector_g(grad)
                    @printf "\n"                
                end
            end
        end
    end

    trajectory = trajectory[:,1:i]; cpm_traj = cpm_traj[:,1:i]
    return vector_wrap(bbox, args, params), trajectory, cost, cpm_traj
end


bbox_Hessian_keyword_minimization

In [ ]:
bbox_Hessian_keyword_minimization()

### Looking at the distribution of VR - VL

In [ ]:
ntrials = 500
proVs, antiVs = @time(run_ntrials(ntrials; plot_list=[], model_params...))

@printf("Pro %% correct = %g%%\n", 100*length(find(proVs[1,:].>proVs[4,:]))/ntrials)
@printf("Anti %% correct = %g%% \n", 100*length(find(antiVs[1,:].<antiVs[4,:]))/ntrials)

figure(1); clf();
ax1 = subplot(2,1,1)
h = plt[:hist](proVs[1,:]-proVs[4,:],-0.1:0.002:0.1)
title("PRO Vr - Vl")
remove_xtick_labels(ax1)
vlines(0, ylim()[1], ylim()[2])

ax2 = subplot(2,1,2)
h = plt[:hist](antiVs[1,:]-antiVs[4,:],-0.1:0.002:0.1)
title("ANTI Vr - Vl")
vlines(0, ylim()[1], ylim()[2])
